In [1]:
import pandas as pd
import os, re, string
# from langdetect import detect
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Checkout\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Checkout\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Import csv files dynamically
current_directory = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(current_directory, '..'))
job_posting = os.path.join(project_root, 'src', 'data', 'job_posting_linkedin.csv')
students = os.path.join(project_root, 'src', 'data', 'students.csv')

In [20]:
job_posting = pd.read_csv(job_posting)
students = pd.read_csv(students)

In [5]:
job_posting.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.0,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift,"Metalcraft of Mayville, Inc.","Management, Manufacturing","Headquartered in Mayville, Wisconsin, Metalcra...",Industrial Machinery Manufacturing,"Metal Fabrication, Machining, Robotic Welding,...",Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",OffsiteApply,FULL_TIME,USD,NaN,WI,Mayville,US,1000 Metalcraft Drive,300.0,https://www.linkedin.com/company/metalcraft-of...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.5,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb


In [12]:
job_posting.isnull().sum()

job_id                     0
company_id                 0
title                      7
company_name               4
job_skills              1016
company_description      189
company_industry          72
company_speciality      6609
description                0
max_salary             30334
med_salary             30334
min_salary             30334
pay_period             19429
formatted_work_type        0
location                   0
application_type           0
work_type                  0
currency                   0
compensation_type      19429
state                     11
city                       1
country                    0
address                   17
employee_count             0
linkedin_url               0
dtype: int64

In [13]:
job_posting[job_posting['description'].isnull()]

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url


In [12]:
# if description is empty fill it with company_description, company_industry and job_skills
job_posting['description'] = job_posting['description'].fillna(job_posting['company_description'] + ' ' + job_posting['company_industry'] + ' ' + job_posting['job_skills'])

In [14]:
contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "here's": "here is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    "u.s": "america",
    "e.g": "for example",
}

In [15]:
# Clean contraction
def clean_contractions(text):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in contraction_mapping.keys():
        if "" + word + "" in text:
            text = text.replace("" + word + "", "" + contraction_mapping[word] + "")
    return text

In [16]:
# Remove Url Pattern
def remove_urls(text):
    url_pattern = r"https?://\S+|www\.\S+"
    return re.sub(url_pattern, "", text)

In [17]:
# Remove HTML Tag
def remove_html(text):
    html_pattern = re.compile("<.*?>")
    return html_pattern.sub(r"", text)

In [18]:
# Remove special character "!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"
def remove_punctuation(text):
    punctuation = string.punctuation + "–"
    return re.sub(f"[{re.escape(punctuation)}]", "", text)

In [19]:
# Remove E-mail pattern
def remove_emails(text):
    return re.sub(r"\S+@\S+", "", text)

In [20]:
# Remove New Line Code Snippet
def remove_code_snippet(text):
    return text.replace("\n", " ")

In [21]:
# Remove Emoji
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

In [22]:
# Remove Non-English Text
def remove_non_english_text(text):
    def is_english(text):
        try:
            return detect(text) == "en"
        except:
            return False

    return text if is_english(text) else ""

In [23]:
# Remove Digits
def remove_digits(text):
    return "".join(filter(lambda char: not char.isdigit(), text))

In [24]:
# Remove Stop Words
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    custom_stop_words = [
        "job",
        "role",
        "position",
        "responsibility",
        "responsibilities",
        "duties",
        "duty",
        "requirement",
        "requirements",
        "qualification",
        "qualifications",
        "description",
        "descriptions",
        "candidate",
        "candidates",
        "applicant",
        "applicants",
        "opportunity",
        "opportunities",
        "team",
        "teams",
        "work",
        "working",
        "employee",
        "employees",
        "employer",
        "employers",
        "company",
        "companies",
        "location",
        "locations",
        "department",
        "departments",
        "report",
        "reports",
        "reporting",
        "benefit",
        "benefits",
        "compensation",
        "salary",
        "experience",
        "experienced",
        "year",
        "years",
        "gender",
        "race",
        "color",
        "sex",
        "orientation",
        "sexual",
        "religion",
        "national",
        "identify",
        "veteran",
        "nation",
        "including",
        "required",
        "disability",
        "regard"
    ]

In [25]:
# Remove Stop Words
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    custom_stop_words = [
        "job",
        "role",
        "position",
        "responsibility",
        "responsibilities",
        "duties",
        "duty",
        "requirement",
        "requirements",
        "qualification",
        "qualifications",
        "description",
        "descriptions",
        "candidate",
        "candidates",
        "applicant",
        "applicants",
        "opportunity",
        "opportunities",
        "team",
        "teams",
        "work",
        "working",
        "employee",
        "employees",
        "employer",
        "employers",
        "company",
        "companies",
        "location",
        "locations",
        "department",
        "departments",
        "report",
        "reports",
        "reporting",
        "benefit",
        "benefits",
        "compensation",
        "salary",
        "experience",
        "experienced",
        "year",
        "years",
        "gender",
        "race",
        "color",
        "sex",
        "orientation",
        "sexual",
        "religion",
        "national",
        "identify",
        "veteran",
        "nation",
        "including",
        "required",
        "disability",
        "regard"
    ]

    words = text.split()
    filtered_words = [
        word
        for word in words
        if word.lower() not in stop_words and word.lower() not in custom_stop_words
    ]
    return " ".join(filtered_words)

In [26]:
# Lemmatization
def lemmatize_words(text):
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

In [28]:
def clean_text(text,remove_stop_words=True):
    text = text.lower()
    text = clean_contractions(text)
    text = remove_urls(text)
    text = remove_html(text)
    text = remove_code_snippet(text)
    text = remove_punctuation(text)
    text = remove_emails(text)
    text = remove_emoji(text)
    text = remove_non_english_text(text)
    text = remove_digits(text)
    text = lemmatize_words(text)
    if remove_stop_words:
        text = remove_stopwords(text)
    return text

In [29]:
#Clean Job description text\
job_posting['description_cleaned'] = job_posting['description'].astype(str).apply(lambda x: clean_text(x))
job_posting['description_cleaned_st'] = job_posting['description'].astype(str).apply(lambda x: clean_text(x, remove_stop_words=False))
job_posting['company_description_cleaned'] = job_posting['company_description'].astype(str).apply(lambda x: clean_text(x, remove_stop_words=False))

In [30]:
job_posting.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.0,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife hearing care part demant ...,overview hearinglife is a national hearing car...,hearinglifeufis a national hearing care compan...
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift,"Metalcraft of Mayville, Inc.","Management, Manufacturing","Headquartered in Mayville, Wisconsin, Metalcra...",Industrial Machinery Manufacturing,"Metal Fabrication, Machining, Robotic Welding,...",Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",OffsiteApply,FULL_TIME,USD,NaN,WI,Mayville,US,1000 Metalcraft Drive,300.0,https://www.linkedin.com/company/metalcraft-of...,metalcraft mayville metalcraft mayville equal ...,metalcraft of mayville metalcraft of mayville ...,headquartered in mayville wisconsin metalcraft...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,tsubaki name synonymous excellence quality dep...,the tsubaki name is synonymous with excellence...,america tsubaki power transmission llc is a su...
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.5,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...,episcopal community service ec ha become the p...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,summary iherb mission make health wellness acc...,job summary at iherb we are on a mission to ma...,iherb is on a mission to make health and welln...


In [32]:
job_posting['job_posting_description'] = job_posting['description_cleaned_st'] + ' ' + job_posting['company_industry'] + ' ' + job_posting['job_skills'] + ' ' + job_posting['company_speciality'] + ' ' + job_posting['company_description_cleaned']

In [33]:
# you can run this cell to get final job_posting file
job_posting.to_csv('job_posting_description_final1.csv', index=False)

In [7]:
df = pd.read_csv('job_posting_description_final1.csv')

In [4]:
df['description_cleaned_st'].isnull().sum()

46

In [5]:
df['company_industry'].isnull().sum()

72

In [6]:
df['job_skills'].isnull().sum()

1016

In [7]:
df['company_speciality'].isnull().sum()

6609

In [8]:
df['company_description_cleaned'].isnull().sum()

326

In [8]:
filter1 = df[df['job_posting_description'].isnull()]
filter1.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.50,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...,episcopal community service ec ha become the p...,NaN
6,3757937004,10515052,Auto Body Techncian,Boyd Group Services Inc.,"Management, Manufacturing",Boyd Group Services Inc. (”BGSI”) is a Canadia...,Automotive,NaN,Company: Gerber Collision & Glass\n\nWELCOME T...,NaN,NaN,NaN,NaN,Full-time,"Daytona Beach, FL",OffsiteApply,FULL_TIME,USD,NaN,Manitoba,Winnipeg,CA,1745 Ellice Ave,244.0,https://www.linkedin.com/company/the-boyd-grou...,gerber collision glass welcome gerber collisio...,company gerber collision glass welcome to gerb...,boyd group service inc ”bgsi” is a canadian co...,NaN
7,3757936167,2915,ACME D8,Acme Markets,"Sales, Business Development",Your Favorite Local Supermarket\n\nCelebrating...,Retail,NaN,The First Assistant Store Director is actively...,NaN,NaN,NaN,NaN,Full-time,"Sussex, NJ",OffsiteApply,FULL_TIME,USD,NaN,Pennsylvania,Malvern,US,75 Valley Stream Pkwy,4039.0,https://www.linkedin.com/company/acme-markets,first assistant store director actively involv...,the first assistant store director is actively...,your favorite local supermarketnncelebrating y...,NaN
8,3757936097,18213359,Dishwasher,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",21.23,19.30,17.37,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle signon bonus guaranteed looki...,descriptiontitle signon bonus guaranteed are y...,episcopal community service ec ha become the p...,NaN
14,3757935025,99500291,Associate Veterinarian,Vet Doc Hospitals,Health Care Provider,Compassionate Care for Your Furry Friends nati...,Veterinary Services,NaN,We are looking to hire an Associate Veterinari...,NaN,NaN,NaN,NaN,Full-time,"St Clair Shores, Michigan, United States",OffsiteApply,FULL_TIME,USD,NaN,Pennsylvania,Philadelphia,US,0,1.0,https://www.linkedin.com/company/vetdochospitals,looking hire associate veterinarian join pract...,we are looking to hire an associate veterinari...,compassionate care for your furry friend natio...,NaN


In [13]:
# Concatenate the values from the specified columns
concatenated_values = (
    df['description_cleaned_st'].fillna('') + ' ' +
    df['company_industry'].fillna('') + ' ' +
    df['job_skills'].fillna('') + ' ' +
    df['company_speciality'].fillna('') + ' ' +
    df['company_description_cleaned'].fillna('')
)

# Fill null values in the job_posting_description column with the concatenated values
df['job_posting_description'] = df['job_posting_description'].fillna(concatenated_values)

In [14]:
df[df['job_posting_description'].isnull()]

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32545 entries, 0 to 32544
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_id                       32545 non-null  int64  
 1   company_id                   32545 non-null  int64  
 2   title                        32538 non-null  object 
 3   company_name                 32541 non-null  object 
 4   job_skills                   31529 non-null  object 
 5   company_description          32356 non-null  object 
 6   company_industry             32473 non-null  object 
 7   company_speciality           25936 non-null  object 
 8   description                  32545 non-null  object 
 9   max_salary                   2211 non-null   float64
 10  med_salary                   2211 non-null   float64
 11  min_salary                   2211 non-null   float64
 12  pay_period                   13116 non-null  object 
 13  formatted_work_t

In [17]:
# you can run this cell to get final job_posting file
df.to_csv('job_posting_description_final2.csv', index=False)

In [3]:
import numpy as np
df = pd.read_csv('job_posting_description_final2.csv')

In [4]:
df.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.0,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife hearing care part demant ...,overview hearinglife is a national hearing car...,hearinglifeufis a national hearing care compan...,overview hearinglife is a national hearing car...
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift,"Metalcraft of Mayville, Inc.","Management, Manufacturing","Headquartered in Mayville, Wisconsin, Metalcra...",Industrial Machinery Manufacturing,"Metal Fabrication, Machining, Robotic Welding,...",Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",OffsiteApply,FULL_TIME,USD,NaN,WI,Mayville,US,1000 Metalcraft Drive,300.0,https://www.linkedin.com/company/metalcraft-of...,metalcraft mayville metalcraft mayville equal ...,metalcraft of mayville metalcraft of mayville ...,headquartered in mayville wisconsin metalcraft...,metalcraft of mayville metalcraft of mayville ...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,tsubaki name synonymous excellence quality dep...,the tsubaki name is synonymous with excellence...,america tsubaki power transmission llc is a su...,the tsubaki name is synonymous with excellence...
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.5,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...,episcopal community service ec ha become the p...,descriptiontitle looking for a great opportuni...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,summary iherb mission make health wellness acc...,job summary at iherb we are on a mission to ma...,iherb is on a mission to make health and welln...,job summary at iherb we are on a mission to ma...


In [5]:
df.columns

Index(['job_id', 'company_id', 'title', 'company_name', 'job_skills',
       'company_description', 'company_industry', 'company_speciality',
       'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period',
       'formatted_work_type', 'location', 'application_type', 'work_type',
       'currency', 'compensation_type', 'state', 'city', 'country', 'address',
       'employee_count', 'linkedin_url', 'description_cleaned',
       'description_cleaned_st', 'company_description_cleaned',
       'job_posting_description'],
      dtype='object')

In [7]:
df = df[['job_id', 'company_id', 'title', 'company_name', 'job_skills', 'max_salary', 'med_salary', 'min_salary', 'pay_period',
       'formatted_work_type', 'location', 'application_type', 'work_type',
       'currency', 'compensation_type', 'state', 'city', 'country', 'address',
       'employee_count', 'linkedin_url',
       'job_posting_description']]

In [9]:
df.to_csv('job_posting_final.csv', index=False)

In [58]:
df['salary'] = np.where(
    (df['min_salary'].notna()) & (df['max_salary'].notna()) & (df['pay_period'].notna()),
    '$' + df['min_salary'].astype(str) + ' - ' + '$' + df['max_salary'].astype(str) + ' ' + df['pay_period'],
    ''  # or '' if you prefer to have empty string instead of NaN for rows not meeting the condition
)

In [59]:
df.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description,salary
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.0,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife hearing care part demant ...,overview hearinglife is a national hearing car...,hearinglifeufis a national hearing care compan...,overview hearinglife is a national hearing car...,$4725.0 - $5775.0 MONTHLY
1,3757940025,2192142,Shipping & Receiving Associate 2nd shift,"Metalcraft of Mayville, Inc.","Management, Manufacturing","Headquartered in Mayville, Wisconsin, Metalcra...",Industrial Machinery Manufacturing,"Metal Fabrication, Machining, Robotic Welding,...",Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",OffsiteApply,FULL_TIME,USD,NaN,WI,Mayville,US,1000 Metalcraft Drive,300.0,https://www.linkedin.com/company/metalcraft-of...,metalcraft mayville metalcraft mayville equal ...,metalcraft of mayville metalcraft of mayville ...,headquartered in mayville wisconsin metalcraft...,metalcraft of mayville metalcraft of mayville ...,
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,tsubaki name synonymous excellence quality dep...,the tsubaki name is synonymous with excellence...,america tsubaki power transmission llc is a su...,the tsubaki name is synonymous with excellence...,
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.5,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...,episcopal community service ec ha become the p...,descriptiontitle looking for a great opportuni...,$20.04 - $24.5 HOURLY
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,summary iherb mission make health wellness acc...,job summary at iherb we are on a mission to ma...,iherb is on a mission to make health and welln...,job summary at iherb we are on a mission to ma...,


In [60]:
f = df[df['salary'] != '']
f.head(50)

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description,salary
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.00,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife hearing care part demant ...,overview hearinglife is a national hearing car...,hearinglifeufis a national hearing care compan...,overview hearinglife is a national hearing car...,$4725.0 - $5775.0 MONTHLY
3,3757938018,18213359,Cook,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\n Looking for a great oppor...,24.50,22.27,20.04,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...,episcopal community service ec ha become the p...,descriptiontitle looking for a great opportuni...,$20.04 - $24.5 HOURLY
8,3757936097,18213359,Dishwasher,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",21.23,19.30,17.37,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle signon bonus guaranteed looki...,descriptiontitle signon bonus guaranteed are y...,episcopal community service ec ha become the p...,descriptiontitle signon bonus guaranteed are y...,$17.37 - $21.23 HOURLY
23,3757934423,18213359,Custodian/Janitor,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,descriptionTitle\n\nCome work as a Custodian a...,21.23,19.30,17.37,HOURLY,Full-time,"Altadena, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle come custodian great communit...,descriptiontitle come work a a custodian at a ...,episcopal community service ec ha become the p...,descriptiontitle come work a a custodian at a ...,$17.37 - $21.23 HOURLY
69,3757931788,6464,Admissions Processing Supervisor,University of North Texas,"Accounting/Auditing, Finance","Established in 1890, UNT is one of the nation’...",Higher Education,"business, education, higher education, enginee...",Position Details\n\nPosition Information\n\nUN...,44000.00,40000.00,36000.00,YEARLY,Full-time,"Denton, TX",OffsiteApply,FULL_TIME,USD,BASE_SALARY,Tx,Denton,US,1155 Union Circle #311277,11624.0,https://www.linkedin.com/school/northtexas/,detail information unt system overview welcome...,position detail position information unt syste...,established in unt is one of the nation larges...,position detail position information unt syste...,$36000.0 - $44000.0 YEARLY
98,3757929967,6577380,CDL Class A Driver,Conexwest,"Management, Manufacturing",Conexwest is the leading supplier of standard ...,Truck Transportation,"storage containers, office containers, rent le...",We are looking for punctual drivers with good ...,30.80,28.00,25.20,HOURLY,Full-time,"Oakland, CA",OffsiteApply,FULL_TIME,USD,BASE_SAL

In [61]:
# Mask for company + title dups
mask_1 = df.duplicated(["company_id", "title"], keep=False)

# Mask for company + title + description dups
mask_2 = df.duplicated(["company_id", "title", "description"], keep=False)

print(f"Rows duplicated for 'company_id' + 'title': {df.loc[mask_1].shape[0]}")
print(f"Rows duplicated for 'company_id' + 'title' + 'description': {df.loc[mask_2].shape[0]}")

Rows duplicated for 'company_id' + 'title': 8645
Rows duplicated for 'company_id' + 'title' + 'description': 4088


In [62]:
# Removing duplicated company_id + title + description
print(f"Rows before: {df.shape[0]}")
df = df.loc[df.duplicated(["company_id", "title", "description"], keep="first") == False].copy(deep=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 32545
Rows after: 29674


In [63]:
from difflib import SequenceMatcher
import pandas as pd

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [64]:
# Applying the string similarity to the descriptions
# The idea is to identify descriptions that are not exactly the same
# but have a big overlap which could be eliminated

# The logic
# First fetch all the duplicates (except for the 'description' column).
# Then apply a mask to identify the first occurence of each subset of duplicates.
# With the new column 'first_dup' we can create a nummeric column which will serve as index.
# To create the index we can use the pandas cumsum() method
# Lastly we extract only the first occurences to compare with the copies

# All duplicates
dups = df[df.duplicated(["company_id", "title"], keep=False)].sort_values(["company_id", "title"]).copy(deep=True)

# Add a column for first
dups["first_dup"] = ~dups.duplicated(["company_id", "title"], keep="first") # bool col
dups["first_dup_num"] = dups["first_dup"].astype(int) # True -> 1, False -> 0
dups["dup_idx"] = dups["first_dup_num"].cumsum()

# Subset the first occurences only and rename the description column
dups_first = dups[dups["first_dup"] == True].copy(deep=True)
dups_first = dups_first[["dup_idx", "description"]].copy(deep=True)
dups_first.rename(columns={"description": "first_description"}, inplace=True)

# Merge and apply the similarity function
dups = pd.merge(left=dups, 
                right=dups_first, 
                left_on="dup_idx",
                right_on="dup_idx",
                how="left", # left or inner will provide same results
                validate="many_to_one") # use validate to make sure

dups["similarity_ratio"] = dups.apply(lambda x: similarity(x.description, x.first_description), axis=1)
dups.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description,salary,first_dup,first_dup_num,dup_idx,first_description,similarity_ratio
0,3756109330,1016,Biomedical Technician 2,GE HealthCare,"Management, Manufacturing",Every day millions of people feel the impact o...,"Hospitals and Health Care, Medical Equipment M...","Healthcare, Biotechnology",Job Description Summary \n\nAs the Biomedical ...,NaN,NaN,NaN,NaN,Full-time,"Georgia, United States",SimpleOnsiteApply,FULL_TIME,USD,NaN,0,Chicago,US,-,"53495.0, 54640.0, 54639.0",https://www.linkedin.com/company/gehealthcare,summary biomedical technician respond service ...,job description summary a the biomedical techn...,every day million of people feel the impact of...,job description summary a the biomedical techn...,,True,1,1,Job Description Summary \n\nAs the Biomedical ...,1.000000
1,3756107560,1016,Biomedical Technician 2,GE HealthCare,"Management, Manufacturing",Every day millions of people feel the impact o...,"Hospitals and Health Care, Medical Equipment M...","Healthcare, Biotechnology",Job Description Summary \n\nAs the Biomedical ...,NaN,NaN,NaN,NaN,Full-time,"Massachusetts, United States",SimpleOnsiteApply,FULL_TIME,USD,NaN,0,Chicago,US,-,"53495.0, 54640.0, 54639.0",https://www.linkedin.com/company/gehealthcare,summary biomedical technician respond service ...,job description summary a the biomedical techn...,every day million of people feel the impact of...,job description summary a the biomedical techn...,,False,0,1,Job Description Summary \n\nAs the Biomedical ...,0.997369
2,3756107522,1016,Field Engineer 1,GE HealthCare,"Engineering, Information Technology",Every day millions of people feel the impact o...,"Hospitals and Health Care, Medical Equipment M...","Healthcare, Biotechnology",Job Description Summary \n\nThe Field Engineer...,NaN,NaN,NaN,NaN,Full-time,"Kansas, United States",OffsiteApply,FULL_TIME,USD,NaN,0,Chicago,US,-,"53495.0, 54640.0, 54639.0",https://www.linkedin.com/company/gehealthcare,summary field engineer perform preventative ma...,job description summary the field engineer wil...,every day million of people feel the impact of...,job description summary the field engineer wil...,,True,1,2,Job Description Summary \n\nThe Field Engineer...,1.000000
3,3699082102,1016,Field Engineer 1,GE HealthCare,"Engineering, Information Technology",Every day millions of people feel the impact o...,"Hospitals and Health Care, Medical Equipment M...","Healthcare, Biotechnology",Job Description Summary \n\nThe Field Engineer...,NaN,NaN,NaN,YEARLY,Full-time,"Delaware, United States",OffsiteApply,FULL_TIME,USD,BASE_SALARY,0,Chicago,US,-,"53495.0, 54640.0, 54639.0",https://www.linkedin.com/company/gehealthcare,summary field engineer perform preventative ma...,job description summary the field engineer wil...,every day million of people feel the impact of...,job description summary the field engineer wil...,,False,0,2,Job Description Summary \n\nThe Field Engineer...,0.933188
4,3748850229,1028,Senior Software Engineer,Oracle,"Engineering, Information Technology","""We’re a cloud technology company that provide...",IT Services and IT Consulting,"enterprise, software, applications, database, ...","Oracle, Full Time, Remote, United StatesSenior...",NaN,NaN,NaN,NaN,Full-time,United States,ComplexOnsiteApply,FULL_TIME,USD,NaN,Texas,Austin,US,2300 Oracle Way,"202019.0, 202050.0, 200515.0, 200654.0, 200714...",https://www.linkedin.com/company/oracle,oracle full time remote united statessenior so...,oracle full time remote united statessenior so...,we are a cloud technology company that provide...,oracle full time remote united statessenior so...,,True

In [66]:
# Removing the similar texts
print(f"Rows before: {df.shape[0]}")
df = df.loc[(~df["job_id"].isin(to_drop["job_id"])) & (~df["company_id"].isin(to_drop["company_id"]))]
print(f"Rows after: {df.shape[0]}")

Rows before: 29674
Rows after: 21823


In [67]:
# Free-up space (we need to handle Kaggle's memory finely lol...)
del dups, dups_first, to_drop

In [68]:
df['title'] = df['title'].str.title()

In [69]:
df[df['company_name'] == 'BURGEON IT SERVICES']

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description,salary
3357,3757704955,1850810,Azure Kubernetes Service Admin,BURGEON IT SERVICES,Administrative,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Role: Azure Kubernetes Service (AKS) AdminLoca...,NaN,NaN,NaN,NaN,Contract,"Brentwood, TN",ComplexOnsiteApply,CONTRACT,USD,NaN,Delaware,Claymont,US,"619,","239.0, 245.0",https://www.linkedin.com/company/burgeon-it-se...,azure kubernetes service ak adminlocation bren...,role azure kubernetes service ak adminlocation...,burgeon is a provider of technical and it staf...,role azure kubernetes service ak adminlocation...,
4178,3757493152,1850810,Aws Solution Architect,BURGEON IT SERVICES,NaN,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Role: AWS Cloud solution architectLocation : F...,NaN,NaN,NaN,NaN,Full-time,"Franklin, MA",SimpleOnsiteApply,FULL_TIME,USD,NaN,Delaware,Claymont,US,"619,","239.0, 245.0",https://www.linkedin.com/company/burgeon-it-se...,aws cloud solution architectlocation franklin ...,role aws cloud solution architectlocation fran...,burgeon is a provider of technical and it staf...,role aws cloud solution architectlocation fran...,
6142,3757465896,1850810,Role: Aws Cloud Solution Architect At Onsite,BURGEON IT SERVICES,Information Technology,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Role: AWS Cloud solution architectLocation : F...,NaN,NaN,NaN,NaN,Full-time,"Franklin, MA",ComplexOnsiteApply,FULL_TIME,USD,NaN,Delaware,Claymont,US,"619,","239.0, 245.0",https://www.linkedin.com/company/burgeon-it-se...,aws cloud solution architectlocation franklin ...,role aws cloud solution architectlocation fran...,burgeon is a provider of technical and it staf...,role aws cloud solution architectlocation fran...,
6354,3757463007,1850810,Role:,BURGEON IT SERVICES,Information Technology,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Role: Salesforce Business System AnalystLocati...,NaN,NaN,NaN,NaN,Contract,United States,ComplexOnsiteApply,CONTRACT,USD,NaN,Delaware,Claymont,US,"619,","239.0, 245.0",https://www.linkedin.com/company/burgeon-it-se...,salesforce business system analystlocation rem...,role salesforce business system analystlocatio...,burgeon is a provider of technical and it staf...,role salesforce business system analystlocatio...,
7386,3757446819,1850810,.Net Architect,BURGEON IT SERVICES,Information Technology,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Job Title: .Net Architect Location: REMOTEJD:S...,NaN,NaN,NaN,NaN,Contract,San Francisco Bay Area,ComplexOnsiteApply,CONTRACT,USD,NaN,Delaware,Claymont,US,"619,","239.0, 245.0",https://www.linkedin.com/company/burgeon-it-se...,title net architect remotejdsso deeplinksdata ...,job title net architect location remotejdsso a...,burgeon is a provider of technical and it staf...,job title net architect location remotejdsso a...,
19081,3701309575,1850810,Salesforce Technical Lead With Sales Cloud Exp...,BURGEON IT SERVICES,Information Technology,Burgeon is a provider of technical and IT staf...,IT Services and IT Consulting,"IT Consulting, Contract Staffing, Permanent St...",Position: Salesforce Technical Lead With Sales...,NaN,NaN,NaN,NaN,Contract,"Newark, CA",ComplexOnsiteApply,CONTRACT,USD,NaN,Delaware,Claymont,US,"619,

In [70]:
# you can run this cell to get final job_posting file
df.to_csv('job_posting_description_final3.csv', index=False)

In [2]:
!pip install geocoder

   ---------------------------------------- 0.0/98.6 kB ? eta -:--:--
   ------------------------------------- -- 92.2/98.6 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 98.6/98.6 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
   ------------------------------- ------- 399.4/491.3 kB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 6.2 MB/s eta 0:00:00


### Preprocessing Job Description final 2

In [136]:
job_posting = pd.read_csv('job_posting_description_final2.csv')

In [137]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32545 entries, 0 to 32544
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_id                       32545 non-null  int64  
 1   company_id                   32545 non-null  int64  
 2   title                        32538 non-null  object 
 3   company_name                 32541 non-null  object 
 4   job_skills                   31529 non-null  object 
 5   company_description          32356 non-null  object 
 6   company_industry             32473 non-null  object 
 7   company_speciality           25936 non-null  object 
 8   description                  32545 non-null  object 
 9   max_salary                   2211 non-null   float64
 10  med_salary                   2211 non-null   float64
 11  min_salary                   2211 non-null   float64
 12  pay_period                   13116 non-null  object 
 13  formatted_work_t

In [138]:
job_posting = job_posting[job_posting['title'].str.len() > 4]

In [139]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31889 entries, 0 to 32544
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_id                       31889 non-null  int64  
 1   company_id                   31889 non-null  int64  
 2   title                        31889 non-null  object 
 3   company_name                 31885 non-null  object 
 4   job_skills                   30887 non-null  object 
 5   company_description          31703 non-null  object 
 6   company_industry             31817 non-null  object 
 7   company_speciality           25385 non-null  object 
 8   description                  31889 non-null  object 
 9   max_salary                   2143 non-null   float64
 10  med_salary                   2143 non-null   float64
 11  min_salary                   2143 non-null   float64
 12  pay_period                   12870 non-null  object 
 13  formatted_work_type  

In [140]:
# job_posting['title'] = job_posting['title'].apply(lambda x: x.title() if x.lower() else x)

In [141]:
job_posting = job_posting[~job_posting['title'].str.contains('\d', regex=True, na=False)]

In [142]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30928 entries, 0 to 32544
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   job_id                       30928 non-null  int64  
 1   company_id                   30928 non-null  int64  
 2   title                        30928 non-null  object 
 3   company_name                 30924 non-null  object 
 4   job_skills                   29954 non-null  object 
 5   company_description          30745 non-null  object 
 6   company_industry             30857 non-null  object 
 7   company_speciality           24617 non-null  object 
 8   description                  30928 non-null  object 
 9   max_salary                   2039 non-null   float64
 10  med_salary                   2039 non-null   float64
 11  min_salary                   2039 non-null   float64
 12  pay_period                   12521 non-null  object 
 13  formatted_work_type  

In [143]:
job_posting.head(100)

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,description_cleaned,description_cleaned_st,company_description_cleaned,job_posting_description
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.00,5250.0,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife hearing care part demant ...,overview hearinglife is a national hearing car...,hearinglifeufis a national hearing care compan...,overview hearinglife is a national hearing car...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,tsubaki name synonymous excellence quality dep...,the tsubaki name is synonymous with excellence...,america tsubaki power transmission llc is a su...,the tsubaki name is synonymous with excellence...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,summary iherb mission make health wellness acc...,job summary at iherb we are on a mission to ma...,iherb is on a mission to make health and welln...,job summary at iherb we are on a mission to ma...
5,3757937037,13727,Territory Manager,ZOLL Medical Corporation,"Sales, Business Development","ZOLL Medical Corporation, an Asahi Kasei Group...",Medical Equipment Manufacturing,"Medical Device and Equipment, EMS/Fire Softwar...","Location: Remote, CT, United States of America...",NaN,NaN,NaN,NaN,Full-time,United States,OffsiteApply,FULL_TIME,USD,NaN,01824,MA,US,269 Mill Rd.,"5308.0, 5395.0, 5394.0",https://www.linkedin.com/company/zoll-medical-...,remote ct united state america cardiac managem...,location remote ct united state of america car...,zoll medical corporation an asahi kasei group ...,location remote ct united state of america car...
6,3757937004,10515052,Auto Body Techncian,Boyd Group Services Inc.,"Management, Manufacturing",Boyd Group Services Inc. (”BGSI”) is a Canadia...,Automotive,NaN,Company: Gerber Collision & Glass\n\nWELCOME T...,NaN,NaN,NaN,NaN,Full-time,"Daytona Beach, FL",OffsiteApply,FULL_TIME,USD,NaN,Manitoba,Winnipeg,CA,1745 Ellice Ave,244.0,https://www.linkedin.com/company/the-boyd-grou...,gerber collision glass welcome gerber collisio...,company gerber collision glass welcome to gerb...,boyd group service inc ”bgsi” is a canadian co...,company gerber collision glass welcome to gerb...
8,3757936097,18213359,Dishwasher,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",21.23,19.3,17.37,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descr

In [144]:
job_posting.drop(['description_cleaned', 'description_cleaned_st', 'company_description_cleaned'], axis=1, inplace=True)

In [145]:
job_posting.head()

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,job_posting_description
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.0,5250.0,4725.0,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife is a national hearing car...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,the tsubaki name is synonymous with excellence...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,job summary at iherb we are on a mission to ma...
5,3757937037,13727,Territory Manager,ZOLL Medical Corporation,"Sales, Business Development","ZOLL Medical Corporation, an Asahi Kasei Group...",Medical Equipment Manufacturing,"Medical Device and Equipment, EMS/Fire Softwar...","Location: Remote, CT, United States of America...",NaN,NaN,NaN,NaN,Full-time,United States,OffsiteApply,FULL_TIME,USD,NaN,01824,MA,US,269 Mill Rd.,"5308.0, 5395.0, 5394.0",https://www.linkedin.com/company/zoll-medical-...,location remote ct united state of america car...
6,3757937004,10515052,Auto Body Techncian,Boyd Group Services Inc.,"Management, Manufacturing",Boyd Group Services Inc. (”BGSI”) is a Canadia...,Automotive,NaN,Company: Gerber Collision & Glass\n\nWELCOME T...,NaN,NaN,NaN,NaN,Full-time,"Daytona Beach, FL",OffsiteApply,FULL_TIME,USD,NaN,Manitoba,Winnipeg,CA,1745 Ellice Ave,244.0,https://www.linkedin.com/company/the-boyd-grou...,company gerber collision glass welcome to gerb...


In [146]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30928 entries, 0 to 32544
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_id                   30928 non-null  int64  
 1   company_id               30928 non-null  int64  
 2   title                    30928 non-null  object 
 3   company_name             30924 non-null  object 
 4   job_skills               29954 non-null  object 
 5   company_description      30745 non-null  object 
 6   company_industry         30857 non-null  object 
 7   company_speciality       24617 non-null  object 
 8   description              30928 non-null  object 
 9   max_salary               2039 non-null   float64
 10  med_salary               2039 non-null   float64
 11  min_salary               2039 non-null   float64
 12  pay_period               12521 non-null  object 
 13  formatted_work_type      30928 non-null  object 
 14  location                 30

In [147]:
job_posting = job_posting[~job_posting['company_name'].isna()]

In [148]:
job_posting = job_posting[~job_posting['company_name'].str.contains(r'\\n|\\t|\\u', regex=True)]


In [149]:
job_posting.head(500)

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,job_posting_description
0,3757940104,553718,Hearing Care Provider,HearingLife,Other,HearingLife\u202fis a national hearing care co...,Medical Equipment Manufacturing,"Digital Hearing Instruments, behind ear, in ea...",Overview\n\nHearingLife is a national hearing ...,5775.00,5250.00,4725.00,MONTHLY,Full-time,"Little River, SC",OffsiteApply,FULL_TIME,USD,BASE_SALARY,New Jersey,Somerset,US,580 Howard Avenue,1171.0,https://www.linkedin.com/company/hearing-life,overview hearinglife is a national hearing car...
2,3757938019,474443,Manager,"U.S. Tsubaki Power Transmission, LLC",Engineering,"U.S. Tsubaki Power Transmission, LLC is a subs...",Automation Machinery Manufacturing,"Roller Chain, Engineering Class Chain, Sprocke...",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",OffsiteApply,FULL_TIME,USD,NaN,Illinois,Wheeling,US,301E Marquardt Drive,314.0,https://www.linkedin.com/company/u.s.-tsubaki-...,the tsubaki name is synonymous with excellence...
4,3757937095,437225,Principal Cloud Security Architect,"iHerb, LLC",Information Technology,"""iHerb is on a mission to make health and well...","Retail, Manufacturing, Technology, Information...","Vitamins, Sports Nutrition, Natural Supplement...","Job Summary\nAt iHerb, we are on a mission to ...",NaN,NaN,NaN,YEARLY,Full-time,United States,OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Irvine,US,17400 Laguna Canyon Rd,"1227.0, 1260.0",https://www.linkedin.com/company/iherb,job summary at iherb we are on a mission to ma...
5,3757937037,13727,Territory Manager,ZOLL Medical Corporation,"Sales, Business Development","ZOLL Medical Corporation, an Asahi Kasei Group...",Medical Equipment Manufacturing,"Medical Device and Equipment, EMS/Fire Softwar...","Location: Remote, CT, United States of America...",NaN,NaN,NaN,NaN,Full-time,United States,OffsiteApply,FULL_TIME,USD,NaN,01824,MA,US,269 Mill Rd.,"5308.0, 5395.0, 5394.0",https://www.linkedin.com/company/zoll-medical-...,location remote ct united state of america car...
6,3757937004,10515052,Auto Body Techncian,Boyd Group Services Inc.,"Management, Manufacturing",Boyd Group Services Inc. (”BGSI”) is a Canadia...,Automotive,NaN,Company: Gerber Collision & Glass\n\nWELCOME T...,NaN,NaN,NaN,NaN,Full-time,"Daytona Beach, FL",OffsiteApply,FULL_TIME,USD,NaN,Manitoba,Winnipeg,CA,1745 Ellice Ave,244.0,https://www.linkedin.com/company/the-boyd-grou...,company gerber collision glass welcome to gerb...
8,3757936097,18213359,Dishwasher,Episcopal Communities & Services,"Management, Manufacturing",Episcopal Communities & Services (ECS) has bec...,Non-profit Organizations,NaN,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",21.23,19.30,17.37,HOURLY,Full-time,"Aliso Viejo, CA",OffsiteApply,FULL_TIME,USD,BASE_SALARY,California,Altadena,US,2212 El Molino Ave,36.0,https://www.linkedin.com/company/episcopal-com...,descriptiontitle signon bonus guaranteed are y...
9,3757936026,634806,Instrumentation Quality Control Representative...,STSI (Staffing Technical Services Inc.),Quality Assurance,"STSI, the definitive nationwide technical plac...",Staffing and Recruiting,"Piping, Mechanical, HVAC, Electrical, I&C, Con...",Instrumentation Quality Control Representative...,NaN,NaN,NaN,NaN,Contract,United States,SimpleOnsiteApply,CONTRACT,USD,NaN,IL,Wheaton,US,528 W. Roosevelt Rd. Suite 101,23.0,https://www.linkedin.com/company/staffing-tech...,instrumentation quality control representative...
10,3757935384,232541,Power Utility Distribution Designer II,"Sigma Technologies, Ltd.","Design, Art/Creative, Information Technology","MISSION\nWith an approach that is honest, syst...",Engineering Services,"Telecommunications Network Design, P

In [150]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30898 entries, 0 to 32544
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_id                   30898 non-null  int64  
 1   company_id               30898 non-null  int64  
 2   title                    30898 non-null  object 
 3   company_name             30898 non-null  object 
 4   job_skills               29925 non-null  object 
 5   company_description      30715 non-null  object 
 6   company_industry         30827 non-null  object 
 7   company_speciality       24595 non-null  object 
 8   description              30898 non-null  object 
 9   max_salary               2038 non-null   float64
 10  med_salary               2038 non-null   float64
 11  min_salary               2038 non-null   float64
 12  pay_period               12511 non-null  object 
 13  formatted_work_type      30898 non-null  object 
 14  location                 30

In [151]:
# job_posting.to_csv('job_posting_data.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'job_posting_data.csv'

In [204]:
job_posting['title'] = job_posting['title'].str.split(r'\(').str[0].str.strip()

In [205]:
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30898 entries, 0 to 32544
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_id                   30898 non-null  int64  
 1   company_id               30898 non-null  int64  
 2   title                    30898 non-null  object 
 3   company_name             30898 non-null  object 
 4   job_skills               29925 non-null  object 
 5   company_description      30715 non-null  object 
 6   company_industry         30827 non-null  object 
 7   company_speciality       24595 non-null  object 
 8   description              30898 non-null  object 
 9   max_salary               2038 non-null   float64
 10  med_salary               2038 non-null   float64
 11  min_salary               2038 non-null   float64
 12  pay_period               12511 non-null  object 
 13  formatted_work_type      30898 non-null  object 
 14  location                 30

In [167]:
job_posting = job_posting[~job_posting['title'].str.contains('Travel ER RN|RBT|CA Only|ACME D8|Unique Remote Opportunity For P&C Pricing Actuary|Alzheimer|Role:', regex=True)]

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,job_posting_description
53,3757932775,79098833,Lead Registered Behavior Technician Lead RBT,Vertivert Inc,Health Care Provider,"""End to End recruitment & Software solutions f...",Hospitals and Health Care,"Software Development, Web Development, Mobile ...",Compensation & Benefits:Signing BonusCompetiti...,NaN,NaN,NaN,NaN,Full-time,"Omaha, NE",ComplexOnsiteApply,FULL_TIME,USD,NaN,California,Fremont,US,Dickenson Common,3.0,https://www.linkedin.com/company/vertivert,compensation benefitssigning bonuscompetitive ...
264,3757916399,808034,Travel ER RN job in Santa Barbara,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya Healthcare, Medical Staffing, Travel Nursi...",Aya Healthcare has an immediate opening for th...,NaN,NaN,NaN,NaN,Part-time,"Santa Barbara, CA",OffsiteApply,PART_TIME,USD,NaN,CA,San Diego,US,5930 Cornerstone Ct #300,"18302.0, 19159.0, 19163.0",https://www.linkedin.com/company/ayahealthcare,aya healthcare ha an immediate opening for the...
297,3757915508,808034,Travel ER RN job in Redlands,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya Healthcare, Medical Staffing, Travel Nursi...",Aya Healthcare has an immediate opening for th...,NaN,NaN,NaN,NaN,Part-time,"Redlands, CA",OffsiteApply,PART_TIME,USD,NaN,CA,San Diego,US,5930 Cornerstone Ct #300,"18302.0, 19159.0, 19163.0",https://www.linkedin.com/company/ayahealthcare,aya healthcare ha an immediate opening for the...
382,3757913558,808034,Travel ER RN job in Everett,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya Healthcare, Medical Staffing, Travel Nursi...",Aya Healthcare has an immediate opening for th...,NaN,NaN,NaN,NaN,Part-time,"Everett, WA",OffsiteApply,PART_TIME,USD,NaN,CA,San Diego,US,5930 Cornerstone Ct #300,"18302.0, 19159.0, 19163.0",https://www.linkedin.com/company/ayahealthcare,aya healthcare ha an immediate opening for the...
470,3757911454,808034,Travel ER RN job in Columbia,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya Healthcare, Medical Staffing, Travel Nursi...",Aya Healthcare has an immediate opening for th...,NaN,NaN,NaN,NaN,Part-time,"Columbia, TN",OffsiteApply,PART_TIME,USD,NaN,CA,San Diego,US,5930 Cornerstone Ct #300,"18302.0, 19159.0, 19163.0",https://www.linkedin.com/company/ayahealthcare,aya healthcare ha an immediate opening for the...
491,3757910904,808034,Travel ER RN job in Phoenix,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya Healthcare, Medical Staffing, Travel Nursi...",Aya Healthcare has an immediate opening for th...,NaN,NaN,NaN,NaN,Part-time,"Phoenix, AZ",OffsiteApply,PART_TIME,USD,NaN,CA,San Diego,US,5930 Cornerstone Ct #300,"18302.0, 19159.0, 19163.0",https://www.linkedin.com/company/ayahealthcare,aya healthcare ha an immediate opening for the...
1230,3757750869,5034592,RBT: In,Center for Social Dynamics,Other,Center for Social Dynamics (CSD) is an organiz...,Hospitals and Health Care,"Autism, Health Care, Children, Behavior therap...",Our Intensive Behavior Unit in North Portland ...,NaN,NaN,NaN,NaN,Full-time,"Portland, OR",OffsiteApply,FULL_TIME,USD,NaN,CA,Alameda,US,"1025 Atlantic Ave, Suite 101","730.0, 726.0, 752.0, 750.0",https://www.linkedin.com/company/center-for-so...,our intensive behavior unit in north portland ...
1686,3757741008,808034,Travel ER RN job in Blythe,Aya Healthcare,Health Care Provider,"""Aya Healthcare is the largest healthcare tale...",Staffing and Recruiting,"Aya

In [212]:
job_posting[job_posting['title'].str.contains(r'@SAP', regex=True)]

,job_id,company_id,title,company_name,job_skills,company_description,company_industry,company_speciality,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,application_type,work_type,currency,compensation_type,state,city,country,address,employee_count,linkedin_url,job_posting_description
